In [1]:
import os

In [2]:
%pwd

'd:\\PROGRAMMING\\text_summerizer\\research'

In [3]:
os.chdir("..")

In [4]:
%pwd

'd:\\PROGRAMMING\\text_summerizer'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: list

In [6]:
from ai_model.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from ai_model.utils.common import read_yaml, create_directories


class ConfigrationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name=config.tokenizer_name
        )

        return data_transformation_config

In [7]:
import os
from transformers import AutoTokenizer
from datasets import load_from_disk


class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)

    def convert_examples_to_features(self, example_batch):
        input_encondings = self.tokenizer(
            example_batch["dialogue"], max_length=1024, truncation=True
        )

        with self.tokenizer.as_target_tokenizer():
            target_encondings = self.tokenizer(
                example_batch["summary"], max_length=128, truncation=True
            )

        return {
            "input_ids": input_encondings["input_ids"],
            "attention_mask": input_encondings["attention_mask"],
            "labels": target_encondings["input_ids"],
        }

    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(
            self.convert_examples_to_features, batched=True
        )
        dataset_samsum_pt.save_to_disk(
            os.path.join(self.config.root_dir, "samsum_dataset")
        )

d:\PROGRAMMING\text_summerizer\.env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
try:
    config = ConfigrationManager()
    data_tranfromation_config = config.get_data_transformation_config()
    data_tranfromation = DataTransformation(config=data_tranfromation_config)
    data_tranfromation.convert()
except Exception as e:
    raise e

[2025-08-17 11:48:24,384: INFO: common: yaml file: config\config.yaml loaded successfuly]
[2025-08-17 11:48:24,388: INFO: common: yaml file: params.yaml loaded successfuly]
[2025-08-17 11:48:24,390: INFO: common: created directory at: artifacts]
[2025-08-17 11:48:24,392: INFO: common: created directory at: artifacts/data_transformation]


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]d:\PROGRAMMING\text_summerizer\.env\Lib\site-packages\transformers\tokenization_utils_base.py:4006: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 86705.60 examples/s]
